In [1]:
import requests
import pandas as pd
from datetime import datetime
import requests
from datetime import datetime

from pprint import pprint as pp

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(encoding='utf-8')

True

In [3]:
# Obtencion de API_TOKEN desde Entorno
API_TOKEN = os.getenv('API_TOKEN')

In [4]:
# Funcion para rellenar dict con datos de Estacion Meteorologica
def fill_list(data, dict, comunity_dict, city):
    name = data['city'].get('name')
    dict['names'].append(name if name else None)

    score = data.get('aqi')
    dict['scores'].append(score if score else None)

    format = '%Y%m%d_%H%M%S'
    date = datetime.now().strftime(format)
    ddate = datetime.strptime(date, format)
    dict['dates'].append(ddate if ddate else None)

    co = data['iaqi'].get('co', {}).get('v')
    dict['co'].append(co if co else None)

    h = data['iaqi'].get('h', {}).get('v')
    dict['humidity'].append(h if h is not None else None)

    no2 = data['iaqi'].get('no2', {}).get('v')
    dict['no2'].append(no2 if no2 else None)

    o3 = data['iaqi'].get('o3', {}).get('v')
    dict['o3'].append(o3 if o3 else None)

    p = data['iaqi'].get('p', {}).get('v')
    dict['p'].append(p if p else None)

    pm10 = data['iaqi'].get('pm10', {}).get('v')
    dict['pm10'].append(pm10 if pm10 else None)

    pm25 = data['iaqi'].get('pm25', {}).get('v')
    dict['pm25'].append(pm25 if pm25 else None)

    so2 = data['iaqi'].get('so2', {}).get('v')
    dict['so2'].append(so2 if so2 else None)

    t = data['iaqi'].get('t', {}).get('v')
    dict['temp'].append(t if t else None)

    w = data['iaqi'].get('w', {}).get('v')
    dict['wind'].append(w if w else None)

    community_found = False
    for key, value in comunity_dict.items():
        if city in value:
            dict['comunity'].append(key)
            community_found = True
            break
    if not community_found:
        dict['comunity'].append("None")

In [5]:
# Funcion Gral para chupar datos de la API y crear DF
# API: https://aqicn.org/api/es/
# API DOC: https://aqicn.org/json-api/doc/

def collect_data(cities, comunity_dict):
    # Dict a rellenar que luego transformaremos en DF
    future_df = {
        'names': [],
        'comunity': [],
        'scores': [],
        'dates': [],
        'co': [],
        'humidity': [],
        'no2': [],
        'o3': [],
        'p': [],
        'pm10': [],
        'pm25': [],
        'so2': [],
        'temp': [],
        'wind': [],
    }
    
    # Bucle por cada estacion de cada ciudad
    for city in cities:
        print(city)
        url = f'https://api.waqi.info/search/?keyword={city}&token={API_TOKEN}'
        response = requests.get(url)
        data = response.json()
        for info in data['data']:
            station = info['station']['url']
            url = f'http://api.waqi.info/feed/{station}/?token={API_TOKEN}'
            response_b = requests.get(url)
            data = response_b.json()
            if 'data' in data:
                fill_list(data['data'], future_df, comunity_dict, city)

    # Crear el DF
    df = pd.DataFrame(future_df)

    # Guardar DF con TimeStamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'../1.2 csvs_raw/cities_stations_{timestamp}.csv'
    df.to_csv(filename, index=False, mode='w', header=True)
    print(f"Datos guardados en {filename}")

In [6]:
# Ciudades a estudiar
cities = ['Madrid',
          'Gran Canaria',
          'Barcelona',
          'Bilbao',
          'Sevilla',
          'Vigo',
          'Coruña',
          'Valencia',
          'Tarragona',
          'Burgos',
          'Malaga',
          'Granada',
          'Toledo',
          'Segovia',
          'Soria',
          "Vitoria",
          "Palencia",
          "Zaragoza"
]

In [7]:
comunity_dict = {
    "Madrid": ["Madrid"],
    "Galicia": ["Coruña", "Vigo"],
    "Pais_Vasco":["Bilbao", "Vitoria"],
    "Castilla_Mancha": ["Toledo", "Guadalajara"],
    "Castilla_Leon": ["Soria", "Segovia", "Palencia", "Burgos"],
    "Andalucia": ["Sevilla", "Malaga", "Granada"],
    "Catalunya": ["Barcelona", "Tarragona"],
    "Com_Valenciana": ["Valencia", "Alicante"],
    "Canarias": ["Gran Canaria"],
    "Aragón": ["Zaragoza"]
}

In [8]:
%%time

# Ejecucion del Script
collect_data(cities, comunity_dict)

Madrid
Gran Canaria
Barcelona
Bilbao
Sevilla
Vigo
Coruña
Valencia
Tarragona
Burgos
Malaga
Granada
Toledo
Segovia
Soria
Vitoria
Palencia
Zaragoza
Datos guardados en ../1.2 csvs_raw/cities_stations_20240701_171159.csv
CPU times: total: 1.25 s
Wall time: 2min 30s
